# Prerequisites

This converter relies on Docker for the abstraction of the environment. You can install Docker from [here](https://docs.docker.com/install/).

The used console commands require Bash.

In [20]:
import os

# Install via pip: pyproj
import sys
!{sys.executable} -m pip install pyproj
from pyproj import Transformer, CRS

import xml.etree.ElementTree as ET

Defaulting to user installation because normal site-packages is not writeable


# Download OpenStreetMaps data for Schleswig-Holstein

In [9]:
# Download from http://download.geofabrik.de/europe/germany/schleswig-holstein-latest.osm.pbf if file does not exist in /input folder
if not os.path.isfile('./input/schleswig-holstein-latest.osm.pbf'):
    os.system('wget http://download.geofabrik.de/europe/germany/schleswig-holstein-latest.osm.pbf -P /input')
    print('Downloaded schleswig-holstein-latest.osm.pbf from http://download.geofabrik.de/europe/germany/schleswig-holstein-latest.osm.pbf')
else :
    print('schleswig-holstein-latest.osm.pbf already exists in /input folder')

# Print file size and MD5 hash
print('MD5 hash of schleswig-holstein-latest.osm.pbf:')
os.system('md5sum ./input/schleswig-holstein-latest.osm.pbf')
print('File size of schleswig-holstein-latest.osm.pbf:')
os.system('ls -lh ./input/schleswig-holstein-latest.osm.pbf')

# Print green coloured success message
print('\033[92m' + 'Download of schleswig-holstein-latest.osm.pbf completed' + '\033[0m')

schleswig-holstein-latest.osm.pbf already exists in /input folder
MD5 hash of schleswig-holstein-latest.osm.pbf:
b5c67d5f036a33d87d8a054048dc139b  ./input/schleswig-holstein-latest.osm.pbf
File size of schleswig-holstein-latest.osm.pbf:
-rw-r--r-- 1 armin armin 136M Jan 20 03:29 ./input/schleswig-holstein-latest.osm.pbf
Download of schleswig-holstein-latest.osm.pbf completed


# Find bounding box for the area to be converted

The bounding box coordinates need to be provided in the OSM coordinate system.
You can find the bounding box coordinates for your area of interest at the following websites:
- [Openstreetmap.org](https://www.openstreetmap.org/export#map=10/54.1870/9.9930): Zoom to your area of interest and copy the coordinates shown on the left side of the screen.
- [boundingbox.klokantech.com](https://boundingbox.klokantech.com/): Select the area of interest with the movable box and use DublinCore on the bottom left side as the format to get the coordinates in an easily readable format.

In [10]:
# Set bounding box for Kiel
top=54.408039
left=10.001019
bottom=54.251083
right=10.228299

# Reduce OSM data to the area of interest and remove unnecessary data using Osmosis

Detailed information about the parameters used can be found in the [OSM Wiki](https://wiki.openstreetmap.org/wiki/Osmosis/Detailed_Usage_0.48).

In [12]:
# Execute Java Osmosis command to extract data from bounding box
os.system('./vendor/osmosis/bin/osmosis --rb file=./input/schleswig-holstein-latest.osm.pbf --bounding-box top=' + str(top) + ' left=' + str(left) + ' bottom=' + str(bottom) + ' right=' + str(right) + ' --used-node --write-xml ./input/kiel.osm')

Jan 21, 2024 5:44:11 PM org.openstreetmap.osmosis.core.Osmosis run
INFO: Osmosis Version 0.46
Jan 21, 2024 5:44:12 PM org.openstreetmap.osmosis.core.Osmosis run
INFO: Preparing pipeline.
Jan 21, 2024 5:44:12 PM org.openstreetmap.osmosis.core.Osmosis run
INFO: Launching pipeline execution.
Jan 21, 2024 5:44:12 PM org.openstreetmap.osmosis.core.Osmosis run
INFO: Pipeline executing, waiting for completion.
Jan 21, 2024 5:44:30 PM org.openstreetmap.osmosis.core.Osmosis run
INFO: Pipeline complete.
Jan 21, 2024 5:44:30 PM org.openstreetmap.osmosis.core.Osmosis run
INFO: Total execution time: 18711 milliseconds.


0

# Convert reduced OSM dataset to MATSim network

## Build Docker image

Information on the deprecation message: https://docs.docker.com/engine/deprecated/#legacy-builder-fallback

In [29]:
os.system('./bin/build.sh')

failed to fetch metadata: fork/exec /usr/local/lib/docker/cli-plugins/docker-buildx: no such file or directory

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/



Sending build context to Docker daemon  106.7MB
Step 1/6 : FROM eclipse-temurin:8-jdk-jammy
 ---> c05a9a5ad465
Step 2/6 : RUN mkdir /osm2matsim
 ---> Using cache
 ---> c7c7813d6879
Step 3/6 : WORKDIR /osm2matsim
 ---> Using cache
 ---> 3216d8abfe80
Step 4/6 : COPY . .
 ---> cb1cde973f73
Step 5/6 : RUN javac -cp vendor/matsim-0.9.0/matsim-0.9.0.jar Osm2matsim.java
 ---> Running in 0efdce8c1937
 ---> Removed intermediate container 0efdce8c1937
 ---> 883e1d5b6ddf
Step 6/6 : ENTRYPOINT ["java", "-cp", "vendor/matsim-0.9.0/matsim-0.9.0.jar:.", "Osm2matsim"]
 ---> Running in 1fdcd386f41c
 ---> Removed intermediate container 1fdcd386f41c
 ---> 0c4d405b916e
Successfully built 0c4d405b916e
Successfully tagged osm2matsim:latest


0

## Convert using the Docker image

In [30]:
os.system('./bin/convert.sh input/kiel.osm output/kiel.xml')

Converting input/kiel.osm into output/kiel.xml


2024-01-21 17:42:31,977  WARN MGC:175 Assuming that coordinates are in longitude first notation, i.e. (longitude, latitude).


2024-01-21 17:42:32,508  INFO Logger:? dataFileCache open start
2024-01-21 17:42:32,847  INFO Config:131 context=[file:/osm2matsim/]
2024-01-21 17:42:32,907  INFO Config:131 context=[file:/osm2matsim/]
2024-01-21 17:42:32,922  INFO OsmNetworkReader:130 Falling back to default values.
2024-01-21 17:42:32,925  INFO MatsimXmlParser:147 starting to parse xml from file input/kiel.osm ...
2024-01-21 17:42:32,941  INFO Counter:61 node 1
2024-01-21 17:42:32,941  INFO Counter:61 node 2
2024-01-21 17:42:32,942  INFO Counter:61 node 4
2024-01-21 17:42:32,944  INFO Counter:61 node 8
2024-01-21 17:42:32,945  INFO Counter:61 node 16
2024-01-21 17:42:32,948  INFO Counter:61 node 32
2024-01-21 17:42:32,953  INFO Counter:61 node 64
2024-01-21 17:42:32,960  INFO Counter:61 node 128
2024-01-21 17:42:32,970  INFO Counter:61 node 256
2024-01-21 17:42:32,989  INFO Counter:61 node 512
2024-01-21 17:42:33,012  INFO Counter:61 node 1024
2024-01-21 17:42:33,032  INFO Counter:61 node 2048


2024-01-21 17:42:32,923  WARN OsmNetworkReader:190 No hierarchy layer specified. Will convert every highway specified by setHighwayDefaults.


2024-01-21 17:42:33,063  INFO Counter:61 node 4096
2024-01-21 17:42:33,116  INFO Counter:61 node 8192
2024-01-21 17:42:33,190  INFO Counter:61 node 16384
2024-01-21 17:42:33,311  INFO Counter:61 node 32768
2024-01-21 17:42:33,457  INFO Counter:61 node 65536
2024-01-21 17:42:33,744  INFO Counter:61 node 131072
2024-01-21 17:42:34,291  INFO Counter:61 node 262144
2024-01-21 17:42:35,362  INFO Counter:61 node 524288
2024-01-21 17:42:35,759  INFO Counter:61 way 1
2024-01-21 17:42:35,760  INFO Counter:61 way 2
2024-01-21 17:42:35,761  INFO Counter:61 way 4
2024-01-21 17:42:35,763  INFO Counter:61 way 8
2024-01-21 17:42:35,765  INFO Counter:61 way 16
2024-01-21 17:42:35,767  INFO Counter:61 way 32
2024-01-21 17:42:35,769  INFO Counter:61 way 64
2024-01-21 17:42:35,775  INFO Counter:61 way 128
2024-01-21 17:42:35,783  INFO Counter:61 way 256
2024-01-21 17:42:35,800  INFO Counter:61 way 512
2024-01-21 17:42:35,843  INFO Counter:61 way 1024
2024-01-21 17:42:35,930  INFO Counter:61 way 2048
2024

2024-01-21 17:42:38,644  WARN LinkImpl:140 [permlanes=0.5 of link id 30 may cause problems]
2024-01-21 17:42:38,644  WARN LinkImpl:142  Future occurences of this logging statement are suppressed.
2024-01-21 17:42:38,702  WARN OsmNetworkReader:539 Could not parse maxspeed tag:For input string: "none". Ignoring it.
2024-01-21 17:42:38,745  WARN OsmNetworkReader:539 Could not parse maxspeed tag:For input string: "walk". Ignoring it.


2024-01-21 17:42:38,714  INFO NetworkImpl:152  link # 8192
2024-01-21 17:42:38,752  INFO NetworkImpl:152  link # 16384
2024-01-21 17:42:38,762  INFO OsmNetworkReader:465 ... done creating the links.
2024-01-21 17:42:38,766  INFO OsmNetworkReader:221 = conversion statistics: ==========================
2024-01-21 17:42:38,766  INFO OsmNetworkReader:222 osm: # nodes read:       624839
2024-01-21 17:42:38,767  INFO OsmNetworkReader:223 osm: # ways read:        8467
2024-01-21 17:42:38,767  INFO OsmNetworkReader:224 MATSim: # nodes created: 9172
2024-01-21 17:42:38,767  INFO OsmNetworkReader:225 MATSim: # links created: 18794
2024-01-21 17:42:38,767  INFO OsmNetworkReader:233 = end of conversion statistics ====================
2024-01-21 17:42:38,768  INFO NetworkCleaner:118 running org.matsim.core.network.algorithms.NetworkCleaner algorithm...
2024-01-21 17:42:38,768  INFO NetworkCleaner:121   checking 9172 nodes and 18794 links for dead-ends...
2024-01-21 17:42:38,822  INFO NetworkCleaner

0

# Convert coordinates in MATSim network to the coordinate system used by the simulation

The conversion is based on the coordinate system set for the TransformationFactory in Osm2matsim.java.
If using the following line, the code below will work without any changes:
```
CoordinateTransformation ct = TransformationFactory.getCoordinateTransformation(TransformationFactory.WGS84, TransformationFactory.WGS84_UTM33N);
```

In [34]:
import sys
import time
from tqdm import tqdm
# Load the XML file
xml_file = "./output/kiel.xml"
tree = ET.parse(xml_file)
root = tree.getroot()

# Define coordinate transformation functions
def epsg32633_to_epsg25832(lon, lat):
    transformer = Transformer.from_crs(CRS.from_epsg(32633), CRS.from_epsg(25832), always_xy=True)
    x, y = transformer.transform(lon, lat)
    return x, y

# Get the total number of nodes
total_nodes = len(root.findall('.//node'))

# Parse and update node coordinates
progress_step = total_nodes // 10  # 10% steps
for i, node in enumerate(tqdm(root.findall('.//node'), total=total_nodes, desc='Converting coordinates')):
    lon = float(node.attrib['x'])
    lat = float(node.attrib['y'])
    x, y = epsg32633_to_epsg25832(lon, lat)
    node.attrib['x'] = str(x)
    node.attrib['y'] = str(y)
    
    # Print progress in 10% steps
    if i % progress_step == 0:
        progress = (i + 1) / total_nodes * 100
        tqdm.write(f"Progress: {progress:.2f}%")

# Save the modified XML file
output_file = "./output/kiel_converted_EPSG25832.xml"
# Ensure custom DOCTYPE is used: <!DOCTYPE network SYSTEM "http://www.matsim.org/files/dtd/network_v2.dtd">
# Otherwise, MATSim will not be able to read the file
tree.write(output_file, encoding='utf-8', xml_declaration=True, doctype='<!DOCTYPE network SYSTEM "http://www.matsim.org/files/dtd/network_v2.dtd">')

print(f"Coordinate conversion completed. Output saved to {output_file}")


Converting coordinates:   0%|          | 5/8883 [00:00<03:40, 40.19it/s]

Progress: 0.01%


Converting coordinates:   5%|▌         | 451/8883 [00:09<02:55, 48.10it/s]

Progress: 5.01%


Converting coordinates:  10%|█         | 894/8883 [00:18<02:55, 45.46it/s]

...

Progress: 89.98%


Converting coordinates:  95%|█████████▌| 8445/8883 [02:54<00:09, 46.78it/s]

Progress: 94.98%


Converting coordinates: 100%|██████████| 8883/8883 [03:04<00:00, 48.23it/s]


Progress: 99.98%
Coordinate conversion completed. Output saved to ./output/kiel_converted_EPSG25832.xml


# Preview network

To preview the network, you can use the MATSim network editor:
``` ./bin/edit_network.sh ```